In [ ]:
# install libs
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00


In [ ]:
!pip install -q sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.4 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset
from datasets import load_dataset

ds = load_dataset("Tohrumi/iwslt15_en-vi_10k")

train-00000-of-00001.parquet:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

eval-00000-of-00001.parquet:   0%|          | 0.00/154k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/188k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [ ]:

from transformers import pipeline

translator = pipeline(model="ngdangkhanh/en-vi-mbart50")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
train_ds = ds["train"]["translation"]
val_ds = ds["eval"]["translation"]
test_ds = ds["test"]["translation"]

In [ ]:
test_ds

[{'en': 'When I was little , I thought my country was the best on the planet , and I grew up singing a song called " Nothing To Envy . "',
  'vi': 'Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên là đất nước tốt nhất trên thế giới và tôi thường hát bài " Chúng ta chẳng có gì phải ghen tị . "'},
 {'en': 'And I was very proud .', 'vi': 'Tôi đã rất tự hào về đất nước tôi .'},
 {'en': 'In school , we spent a lot of time studying the history of Kim Il-Sung , but we never learned much about the outside world , except that America , South Korea , Japan are the enemies .',
  'vi': 'Ở trường , chúng tôi dành rất nhiều thời gian để học về cuộc đời của chủ tịch Kim II- Sung , nhưng lại không học nhiều về thế giới bên ngoài , ngoại trừ việc Hoa Kỳ , Hàn Quốc và Nhật Bản là kẻ thù của chúng tôi .'},
 {'en': 'Although I often wondered about the outside world , I thought I would spend my entire life in North Korea , until everything suddenly changed .',
  'vi': 'Mặc dù tôi đã từng tự hỏi không biết thế

In [ ]:
train_ds = Dataset.from_dict({
    "en": [item["en"] for item in train_ds],
    "vi": [item["vi"] for item in train_ds]
})

val_ds = Dataset.from_dict({
    "en": [item["en"] for item in val_ds],
    "vi": [item["vi"] for item in val_ds]
})

test_ds = Dataset.from_dict({
    "en": [item["en"] for item in test_ds],
    "vi": [item["vi"] for item in test_ds]
})

print(train_ds)
print(val_ds)
print(test_ds)

Dataset({
    features: ['en', 'vi'],
    num_rows: 10000
})
Dataset({
    features: ['en', 'vi'],
    num_rows: 1000
})
Dataset({
    features: ['en', 'vi'],
    num_rows: 1268
})


In [ ]:
test_ds["en"]

['When I was little , I thought my country was the best on the planet , and I grew up singing a song called " Nothing To Envy . "',
 'And I was very proud .',
 'In school , we spent a lot of time studying the history of Kim Il-Sung , but we never learned much about the outside world , except that America , South Korea , Japan are the enemies .',
 'Although I often wondered about the outside world , I thought I would spend my entire life in North Korea , until everything suddenly changed .',
 'When I was seven years old , I saw my first public execution , but I thought my life in North Korea was normal .',
 'My family was not poor , and myself , I had never experienced hunger .',
 "But one day , in 1995 , my mom brought home a letter from a coworker 's sister .",
 'It read , " When you read this , all five family members will not exist in this world , because we haven \'t eaten for the past two weeks .',
 'We are lying on the floor together , and our bodies are so weak we are ready to d

In [ ]:
# greedy search
pred_sentences = translator(test_ds['en'], batch_size=32, num_beams=5, do_sample=False)

In [ ]:
# beam search
pred_sentences = translator(test_ds['en'], batch_size=32, num_beams=5)

In [ ]:
pred_sentences = [pred_sentence['generated_text'] for pred_sentence in pred_sentences]

In [ ]:
pred_sentences[0]

'Khi còn nhỏ , tôi nghĩ đất nước tôi là đất nước tốt nhất trên hành tinh này , và tôi lớn lên , hát một bài hát có tên " Nothing To Envy . "'

In [ ]:
test_ds['vi'][0]

'Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên là đất nước tốt nhất trên thế giới và tôi thường hát bài " Chúng ta chẳng có gì phải ghen tị . "'

In [ ]:
import sacrebleu

# greedy search
bleu_score = sacrebleu.corpus_bleu(pred_sentences, [test_ds['vi']], force=True)
bleu_score

BLEU = 33.27 65.4/40.7/26.5/17.7 (BP = 0.996 ratio = 0.996 hyp_len = 33586 ref_len = 33722)

In [ ]:
import sacrebleu

# beam search
bleu_score = sacrebleu.corpus_bleu(pred_sentences, [test_ds['vi']], force=True)
bleu_score

BLEU = 33.27 65.4/40.7/26.5/17.7 (BP = 0.996 ratio = 0.996 hyp_len = 33586 ref_len = 33722)